In [1]:
%load_ext autoreload
%autoreload 2

from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
#import Image
from PIL import Image
import cv2
import random
import collections
from tqdm import tqdm
import poisson_image_editing as poisson

In [2]:
# dataDir='D:/IMANIP Datasets/COCO2017'
# dataType='val2017'
annFile='H:/Datasets/COCO/annotations/instances_train2014.json'

In [3]:
# initialize COCO api for instance annotations
coco = COCO(annFile)
print(len(coco.getImgIds()))

loading annotations into memory...
Done (t=26.31s)
creating index...
index created!
82783


In [4]:
cats = coco.loadCats(coco.getCatIds())
nms=[(cat['name'],cat['id']) for cat in cats]
print(nms)

[('person', 1), ('bicycle', 2), ('car', 3), ('motorcycle', 4), ('airplane', 5), ('bus', 6), ('train', 7), ('truck', 8), ('boat', 9), ('traffic light', 10), ('fire hydrant', 11), ('stop sign', 13), ('parking meter', 14), ('bench', 15), ('bird', 16), ('cat', 17), ('dog', 18), ('horse', 19), ('sheep', 20), ('cow', 21), ('elephant', 22), ('bear', 23), ('zebra', 24), ('giraffe', 25), ('backpack', 27), ('umbrella', 28), ('handbag', 31), ('tie', 32), ('suitcase', 33), ('frisbee', 34), ('skis', 35), ('snowboard', 36), ('sports ball', 37), ('kite', 38), ('baseball bat', 39), ('baseball glove', 40), ('skateboard', 41), ('surfboard', 42), ('tennis racket', 43), ('bottle', 44), ('wine glass', 46), ('cup', 47), ('fork', 48), ('knife', 49), ('spoon', 50), ('bowl', 51), ('banana', 52), ('apple', 53), ('sandwich', 54), ('orange', 55), ('broccoli', 56), ('carrot', 57), ('hot dog', 58), ('pizza', 59), ('donut', 60), ('cake', 61), ('chair', 62), ('couch', 63), ('potted plant', 64), ('bed', 65), ('dining 

In [5]:
def findAnnotationMatchingCriteria(anns, imgdata, lowerCutoff=0.01, upperCutoff=0.35):
    annotation_count = len(anns)    
    toReturn = []
    indexes = random.sample(range(annotation_count), annotation_count)
    while len(indexes) > 0:
        h,w,c = imgdata.shape
        mask=coco.annToMask(anns[indexes.pop()])
        forgedPixelsCount=collections.Counter(mask.flatten())[1]  
        if forgedPixelsCount>=(h*w*lowerCutoff) and forgedPixelsCount<=(h*w*upperCutoff):
            toReturn = mask
            break
        
    return toReturn

In [6]:
def getAffineTransformedMask(foreground, binarymask):
    indices = np.where(binarymask == 1)
    upper = np.min(indices[0])
    lower = np.max(indices[0])
    left = np.min(indices[1])
    right = np.max(indices[1])
    
    width = right - left
    height= lower - upper
    n = random.randint(10,30)
    hor_right = False if (binarymask.shape[1] - (right+ n +width)) <= 0 else True
    hor_left = False if (left - (n+width)) <= 0 else True 

    side = ""
    if hor_right == True and hor_left == True:
        side=random.sample(["R","L"],1)[0]
        
    elif hor_right == True and hor_left == False:
        side = "R"

    elif hor_right == False and hor_left == True:
        side = "L"
    else:
        return([],[])

    if side == "L":
        S = -(width+n)
    else:
        S = width+n

    v = 0    
    lu = random.randint(0,1)
    if((upper-10)>1 and (binarymask.shape[0]-lower-10) >1):
        if lu == 1:
            v = -random.randint(1,upper-10)
        else:
            v = random.randint(1,binarymask.shape[0]-lower-10)
    elif (upper-10)>1 :
        v = -random.randint(1,upper-10)
    elif (binarymask.shape[0]-lower-10) >1:
        v = random.randint(1,binarymask.shape[0]-lower-10)
    else:
        return([],[])
        
    rows,cols = binarymask.shape
    new_binary_mask=[]
    new_foreground=[]
    M = np.float32([[1,0,S],[0,1,v]])
    transformedForeground = cv2.warpAffine(foreground,M,(cols,rows))
    transformedBinaryMask= cv2.warpAffine(binarymask,M,(cols,rows))
    return(transformedForeground,transformedBinaryMask)
        

In [7]:
import imgaug.augmenters as iaa
from PIL import ImageEnhance
import random

def change_brightness_cv(image):
    aug = iaa.AddToBrightness((-50, 50))
    image = aug.augment_image(image)
    return image

def change_brightness_PIL(image):
    enhancer = ImageEnhance.Brightness(image)
    im_output = enhancer.enhance(random.choice([0.5,0.6,0.7,0.8,0.9,1.1,1.2,1.3,1.4,1.5]))
    return im_output

In [12]:
data = []

catIds = coco.getCatIds()
imgIds = coco.getImgIds()
file_name_list=[((coco.loadImgs(imid)[0])['file_name']) for imid in imgIds]
fake_path='H:/Datasets/COCO/COCO2014_CMFD/fake/'
mask_path='H:/Datasets/COCO/COCO2014_CMFD/mask/'

for imgId, file_name in tqdm(zip(imgIds, file_name_list), total=len(file_name_list)):
    if not os.path.exists(fake_path+file_name[:-4]+'_fake.jpg') and not os.path.exists(mask_path+file_name[:-4]+'_gt.jpg'):
        imgdata = cv2.imread('H:/Datasets/COCO/train2014/'+file_name)
        #Convert to RGB
        b,g,r = cv2.split(imgdata)
        if(np.array_equal(np.array(r),np.array(g)) and np.array_equal(np.array(r),np.array(b))):
            continue #Exclude gray scale images
        imgdata = cv2.merge([r,g,b])

        annIds = coco.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        binarymask = findAnnotationMatchingCriteria(anns, imgdata)
        
        if len(binarymask)==0:
            continue

        foreground=imgdata.copy()        
        foreground[:,:,0]=np.array(imgdata[:,:,0] * binarymask )
        foreground[:,:,1]=np.array(imgdata[:,:,1] * binarymask )
        foreground[:,:,2]=np.array(imgdata[:,:,2] * binarymask )

        new_foreground , new_binarymask = getAffineTransformedMask(foreground, binarymask)
        if(len(new_foreground)==0):
            continue

        background = Image.fromarray(imgdata,'RGB')
        new_binarymask = Image.fromarray(new_binarymask*255)
        
        
        if np.random.rand() < 0.35:
            new_foreground_cv = Image.fromarray(new_foreground,'RGB')
            datas = new_foreground_cv.getdata()
            newData = []
            for item in datas:
                if item[0] == 0 and item[1] == 0 and item[2] == 0:
                    newData.append((0, 0, 0, 0))
                else:
                    newData.append(item)
            new_foreground_cv.putdata(newData)

            source_img = np.array(new_foreground_cv)
            target_img = np.array(background)
            mask_img = np.array(new_binarymask)
            background = poisson.poisson_edit(source_img, np.copy(target_img), mask_img)
            background = Image.fromarray(background)
        else:
            new_foreground_PIL = Image.fromarray(new_foreground,'RGB').convert('RGBA')
            datas = new_foreground_PIL.getdata()
            newData = []
            for item in datas:
                if item[0] == 0 and item[1] == 0 and item[2] == 0:
                    newData.append((0, 0, 0, 0))
                else:
                    newData.append(item)
            new_foreground_PIL.putdata(newData)

            if np.random.rand() < 0.35:
                new_foreground_PIL = change_brightness_PIL(new_foreground_PIL)       
            background.paste(new_foreground_PIL,(0,0),mask=new_foreground_PIL.split()[3])

        # print(source_img.shape, target_img.shape, mask_img.shape)
        # print(type(source_img))
        # plt.figure()
        # plt.imshow(source_img)
        # plt.show()
        # print(type(mask_img))
        # plt.figure()
        # plt.imshow(mask_img)
        # plt.show()
        # print(type(target_img))
        # plt.figure()
        # plt.imshow(target_img)
        # plt.show()
    

        #foreground=foreground.resize((background.size[0],background.size[1]),Image.ANTIALIAS)
        

        # background=background.resize(size=(256,256))
        # new_binarymask=new_binarymask.resize(size=(256,256))
        if os.path.exists(fake_path+file_name[:-4]+'_fake.jpg'):
            print(fake_path+file_name[:-4]+'_fake.jpg')
            continue
        
        background.save(fake_path+file_name[:-4]+'_fake.jpg')
        new_binarymask.save(mask_path+file_name[:-4]+'_gt.jpg')
        data.append({
            "image_path" : fake_path+file_name[:-4]+'_fake.jpg',
            "mask_path" : mask_path+file_name[:-4]+'_gt.jpg'
        })

        # print(type(new_binarymask))
        # plt.figure()
        # plt.imshow(new_binarymask)
        # plt.show()
        # print(type(background))
        # plt.figure()
        # plt.imshow(background)
        # plt.show()
        # break

  0%|          | 33/82783 [00:25<18:01:11,  1.28it/s]


KeyboardInterrupt: 

In [9]:
files = os.listdir('Image_Manipulation_Dataset/COCO2017/val2017')

In [10]:
fake_path='Image_Manipulation_Dataset/COCO2017/COCO_FORGED/fake'
mask_path='Image_Manipulation_Dataset/COCO2017/COCO_FORGED/mask'
not_done = []
for x in tqdm.tqdm(files):
    if not os.path.exists(os.path.join(fake_path, x[:-4]+'_fake.jpg')) and not os.path.exists(os.path.join(mask_path, x[:-4]+'_gt.jpg')):
        not_done.append(x)

100%|██████████| 5000/5000 [00:00<00:00, 9385.00it/s]


In [11]:
len(not_done)

1451